In [1]:
import wandb as wandb
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", "3.0.0")

Reusing dataset cnn_dailymail (/home/jvidakovic/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/3cb851bf7cf5826e45d49db2863f627cba583cbc32342df7349dfe6c38060234)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from transformers import PegasusTokenizerFast

model_name = "google/pegasus-large"
tokenizer = PegasusTokenizerFast.from_pretrained(model_name)

In [8]:
len(dataset["validation"])

13368

In [9]:
from utils import prepare_dataset

eval_dataset = prepare_dataset(
    dataset["validation"]["article"],
    dataset["validation"]["highlights"],
    tokenizer=tokenizer
)


In [10]:
train_dataset = prepare_dataset(
    dataset["train"]["article"],
    dataset["train"]["highlights"],
    tokenizer=tokenizer
)

In [12]:
import wandb

wandb.init(project="pegasus-cnn-fine-tuning", entity="jankovidakovic")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Currently logged in as: jankovidakovic (use `wandb login --relogin` to force relogin)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [14]:
from utils import prepare_model

model, optimizer, scheduler = prepare_model(device="cuda")

In [15]:
from transformers import TrainingArguments, IntervalStrategy

output_dir = "./results"

training_args = TrainingArguments(
    output_dir=output_dir,  # output directory
    per_device_train_batch_size=1,  # batch size per device during training, can increase if memory allows
    per_device_eval_batch_size=1,  # batch size for evaluation, can increase if memory allows
    save_steps=500,  # number of updates steps before checkpoint saves
    save_total_limit=3,  # limit the total amount of checkpoints and deletes the older checkpoints
    evaluation_strategy=IntervalStrategy.STEPS,  # evaluation strategy to adopt during training
    eval_steps=100,  # number of update steps before evaluation
    # warmup_steps=500,  # number of warmup steps for learning rate scheduler
    # weight_decay=0.01,  # strength of weight decay
    logging_dir='./logs',  # directory for storing logs
    logging_steps=10,
    #####
    label_smoothing_factor=0.1,
    max_steps=170000,
    per_gpu_train_batch_size=1,  # total batch size = 2
    gradient_accumulation_steps=128,   # batch_size_orig / batch_size_possible
    report_to=["wandb"],
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    model=model,
    # compute_metrics="hehe"  # i suppose computing rouge on every step is overkill
    optimizers=(optimizer, scheduler),
    args=training_args
)

max_steps is given, it will override any value given in num_train_epochs


In [17]:
trainer.train()

AttributeError: 'PegasusDataset' object has no attribute '_data'